#### Task 1: Simple vector embedding generation

**Objective:**
Generate vector embeddings from text data.

**Task Description:**

- load huggingface embedding model
- embed simple text queries

How to select the right embedding model: [MTEB - Massive Text Embedding Benchmark](https://huggingface.co/blog/mteb)


In [48]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/Users/martin/Documents/DEV/build-your-own-chatbot/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [49]:
text = "This is a test document."

query_vector = embedding_model.embed_query(text)

print(f"Embedding vector length: {len(query_vector)}")
print(query_vector[:10])

Embedding vector length: 768
[-0.04895174130797386, -0.03986186906695366, -0.021562762558460236, 0.009908483363687992, -0.038103967905044556, 0.012684364803135395, 0.0434945784509182, 0.07183391600847244, 0.009748569689691067, -0.006986998952925205]


#### Task 2: Generate embedding vectors with custom dataset

**Objective:**
Load custom dataset, preprocess it and generate vector embeddings.

**Task Description:**

- load and split pdf document "AI_Book.pdf" via langchain document loaders
- use RecursiveCharacterTextSplitter to split documents into chunks
- generate embeddings for single documents

**RecursiveCharacterTextSplitter:**

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is `["\n\n", "\n", " ", ""]`. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

In [77]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re

pdf_doc = "./AI_Book.pdf"

# Create pdf data loader
loader = PyPDFLoader(pdf_doc)

# Load and split documents in chunks
pages_chunked = loader.load_and_split(text_splitter=RecursiveCharacterTextSplitter())

# Function to clean text by removing invalid unicode characters, including surrogate pairs
def clean_text(text):
    # Remove surrogate pairs
    text = re.sub(r'[\ud800-\udfff]', '', text)
    # Optionally remove non-ASCII characters (depends on your use case)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return text

pages_chunked_cleaned = [clean_text(chunk.page_content) for chunk in pages_chunked]

print(pages_chunked[0])

page_content='Aurélien GéronHands-on Machine Learning with
Scikit-Learn, Keras, and
TensorFlow
Concepts, Tools, and Techniques to
Build Intelligent SystemsSECOND EDITION
Boston Farnham Sebastopol Tokyo Beijing Boston Farnham Sebastopol Tokyo Beijing' metadata={'source': './AI_Book.pdf', 'page': 2}


In [78]:
print(pages_chunked[1])

page_content='978-1-492-03264-9
[LSI]Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow
by Aurélien Géron
Copyright © 2019 Aurélien Géron. All rights reserved.
Printed in the United States of America.
Published by O’Reilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472.
O’Reilly books may be purchased for educational, business, or sales promotional use. Online editions are
also available for most titles ( http://oreilly.com ). For more information, contact our corporate/institutional
sales department: 800-998-9938 or corporate@oreilly.com .
Editor:  Nicole Tache
Interior Designer:  David FutatoCover Designer:  Karen Montgomery
Illustrator:  Rebecca Demarest
June 2019:  Second Edition
Revision History for the Early Release
2018-11-05: First Release
2019-01-24: Second Release
2019-03-07: Third Release
2019-03-29: Fourth Release
2019-04-22: Fifth Release
See http://oreilly.com/catalog/errata.csp?isbn=9781492032649  for release details.
The O’Reilly logo 

In [81]:
print(f"Number of text chunks: {len(pages_chunked)}")

Number of text chunks: 507


#### Task 3: Store vector embeddings from pdf document to ChromaDB vector database.

**Objective:**

Store vector embeddings into ChromaDB to store knowledge.

**Task Description:**

- create chromadb client
- create chromadb collection
- create langchain vector db client
- store text document chunks and vector embeddings to vector databases

In [91]:
from langchain_chroma import Chroma
import chromadb
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.HttpClient(
    host="localhost",
    port=8000,
    ssl=False,
    headers=None,
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

collection = client.get_or_create_collection("ai_model_book")

vector_db_from_client = Chroma(
    client=client,
    collection_name="ai_model_book",
    embedding_function=embedding_model,
)

In [92]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(pages_chunked[:50]))]

vector_db_from_client.add_documents(documents=pages_chunked[:50], ids=uuids)

['1367f058-d1a8-4b1d-87c5-6c694b37c024',
 'ecfb75a5-a8fe-4b0f-a760-b5b971f93289',
 '1617d05d-421c-4ff9-8980-c9a572752858',
 'c619c00b-763a-41f9-9212-6cdfa3a469a1',
 '12c39d1a-337a-4b34-8a83-fc297b32c409',
 'b110a4ea-7f1b-427b-a8fb-5cf200e70d57',
 'af50bf1a-6161-4835-b18a-7b288b369bc5',
 '74b1b59c-15a8-4e90-b038-17b48ed6f718',
 '057d9517-ea06-4d62-83ec-9a58a783ccd5',
 '83e1f99e-a326-47c9-a8d9-eddb273175d4',
 '5f962335-1d4f-42cc-a03d-2d8901aa074d',
 '9482461f-15f5-49e5-9349-82b8f9ed3079',
 'cb990c5f-ee6f-40e8-a254-c5dbe6cdfad1',
 '56bac093-bd5e-4276-a5f9-0e45271d698c',
 'd5d9a76a-adae-4601-991e-474182b087fe',
 '580483ae-0e2b-4f2b-91c3-d802c59c6ae3',
 'ebbe8298-5f69-48ea-90b0-3dcdee7a3d12',
 'd3e00077-bda6-47e6-97e0-221dd5782aba',
 'df14f0a2-7af7-4b9f-90e5-66d7095229ae',
 'c299b616-1b8a-4051-9089-36a553090f1a',
 '24de827d-3c7f-4f7d-b669-a88f5cbaf18e',
 '58ddfcce-5a27-4833-8e56-531572c34c48',
 '0e3d8ba6-6fcb-4ff8-8113-1267875a6eb3',
 '0213382d-f4b4-4c7c-81ba-c91dfb5f925f',
 '89ee0568-b9d5-

In [93]:
client.count_collections()

1

In [89]:
# client.delete_collection("ai_model_book")

#### Task 4: Access ChromaDB and perform vector search

**Objective:**

Use query to perform vector search against ChromaDB vector database

**Task Description:**

- define query
- run vector search
- print k=3 most similar documents

In [102]:
results = vector_db_from_client.similarity_search(
    "Types of Machine Learning Systems",
    k=3
)
for res in results:
    print(res.page_content)
    print(res.metadata)
    print("\n")

Types of Machine Learning Systems
There are so many different types of Machine Learning systems that it is useful to
classify them in broad categories based on:
•Whether or not they are trained with human supervision (supervised, unsuper‐
vised, semisupervised, and Reinforcement Learning)
•Whether or not they can learn incrementally on the fly (online versus batch
learning)
•Whether they work by simply comparing new data points to known data points,
or instead detect patterns in the training data and build a predictive model, much
like scientists do (instance-based versus model-based learning)
These criteria are not exclusive; you can combine them in any way you like. For
example, a state-of-the-art spam filter may learn on the fly using a deep neural net‐
work model trained using examples of spam and ham; this makes it an online, model-
based, supervised learning system.
Let’s look at each of these criteria a bit more closely.
Supervised/Unsupervised Learning
Machine Learning systems 